# Dictionary Turing Machine: Implementation and Example
adapted from Boaz's implementation of a standard Turing machine

In [2]:
def xortm(state,sym):
    # sample transition function for the primary tape of a Dictionary Turing Machine
    move = "R"
    if state=="START":  state = "EVEN"
    elif state in ["EVEN","ODD"]:
        if sym=="1":
            # flip state if we see a 1
            state = "EVEN" if state=="ODD" else "ODD"
        elif sym== "∅":
            # at end of tape decide on output based on parity
            state = "OUTPUT1" if state=="ODD" else "OUTPUT0"
            move  = "L"
    # boilerplate
    elif state in ["OUTPUT1","OUTPUT0"]:
        if sym != "▷":
            move, sym  = "L", "∅"
        else:
            move, state = "R" , state[-1]+"ANDSTOP"
    elif state[1:] =="ANDSTOP":
        move, sym = "H", state[0]   
    return state,sym,move

In [28]:
def xortm2(state,sym):
    # sample transition function for the key and value tapes of a Dictionary Turing Machine
    move = "R"
    if state=="START":  state = "EVEN"
    elif state in ["EVEN","ODD"]:
        if sym=="1":
            # flip state if we see a 1
            state = "EVEN" if state=="ODD" else "ODD"
        elif sym== "∅":
            # at end of tape decide on output based on parity
            state = "OUTPUT1" if state=="ODD" else "OUTPUT0"
            move  = "L"
    # boilerplate
    elif state in ["OUTPUT1","OUTPUT0"]:
        if sym != "▷":
            move, sym  = "L", "∅"
        else:
            move, state = "R" , state[-1]+"ANDSTOP"
    elif state[1:] =="ANDSTOP":
        move, sym = "H", state[0]   
    return sym,move

In [18]:
def stringOfTape(tape):
    #takes a tape and strips off the "▷" and "∅" characters
    return tape[1:-1]

In [44]:
class DictionaryTuringMachine:
   
    def __init__(self, transition, ktransition, vtransition):
        self.alphabet = [ "▷", "∅","0", "1" ] 
        self.transition = transition
        self.ktransition = ktransition
        self.vtransition = vtransition
        
    def input(self, x):
        self.tape = [ "▷" ] + [str(a) for a in x ] + [ "∅" ]
        self.state = "START"
        self.head = 0
        
    def key(self, x):
        self.key = [ "▷" ] + [str(a) for a in x ] + [ "∅" ]
        self.khead = 0
        
    def value(self, x):
        self.value = [ "▷" ] + [str(a) for a in x ] + [ "∅" ]
        self.vhead = 0
        
    def table(self, T):
        self.table = T
      
    def run(self):
        while True:
            if self.state == "READ":
                if self.key in self.table:
                    x = self.table[stringOfTape(self.key)]
                    self.value = [ "▷" ] + [str(a) for a in x ] + [ "∅" ]
                else:
                    self.value = [ "▷" ] + [ "∅" ]
            elif self.state == "WRITE":
                self.table[stringOfTape(self.key)] = stringOfTape(self.value)
            
            self.state, self.tape[self.head], move, = self.transition(self.state, self.tape[self.head])
            
            self.key[self.khead], kmove = self.ktransition(self.state, self.key[self.khead])
            
            self.key[self.vhead], vmove = self.vtransition(self.state, self.value[self.vhead])

            if move=="L": self.head = max(0, self.head-1)
            if move=="R":self.head += 1
            if move=="H": break
            if self.head >= len(self.tape): 
                self.tape.append("∅")
            if self.khead >= len(self.key): 
                self.key.append("∅")
            if self.vhead >= len(self.value): 
                self.value.append("∅")
        return self.tape[1]

In [45]:
DTM = DictionaryTuringMachine(xortm, xortm2, xortm2)
DTM.input([0,1,0,1,1,1,1,0,0,1])
DTM.key([0,1,0,1,1,1,1,0,0,1])
DTM.value([0,1,0,1,1,0,0,1,0,1])
DTM.table({'0010':'11'})
DTM.run()

'0'